# Torch Device GPU Setting
- 작성일자: 2019.12.29
- 작성자: MyungHoon Jin (github.com/jinmang2)
- `ChatSpace` 모듈의 업데이트 시점에 따라 해당 내용이 바뀔 수 있습니다.

In [1]:
# spacing
from chatspace import ChatSpace

# If you want to use gpu on torch, set parameter name 'device' as 'cuda'.
spacer = ChatSpace(device='cuda')

Loading JIT Compiled ChatSpace Model


In [105]:
texts = ['안녕안녕나는승배야',
         '어허한두번이아니야경찰서가고싶어?']

In [3]:
spacer.space(texts)

RuntimeError: Expected object of backend CUDA but got backend CPU for argument #3 'index'
The above operation failed in interpreter, with the following stack trace:
at code/model_jit.py:43:17
  bias1 = _22.bias
  _23 = self.batch_normalization
  weight3 = _23.weight
  bias2 = _23.bias
  running_mean = _23.running_mean
  running_var = _23.running_var
  _24 = self.layer_normalization
  weight4 = _24.weight
  bias3 = _24.bias
  embed_input = torch.embedding(weight, input, -1, False, False)
                ~~~~~~~~~~~~~~~ <--- HERE
  input0 = torch.transpose(embed_input, 1, 2)
  input1 = torch._convolution(input0, _2, _3, [1], [0], [1], False, [0], 1, False, False, True)
  input2 = torch.constant_pad_nd(input1, [1, 1], 0)
  input3 = torch._convolution(input2, _5, _6, [1], [0], [1], False, [0], 1, False, False, True)
  conv2_paded = torch.constant_pad_nd(input3, [3, 3], 0)
  input4 = torch.cat([input2, conv2_paded], 1)
  input5 = torch._convolution(input4, _8, _9, [1], [0], [1], False, [0],

In [33]:
# cpu에서는 잘 된다.
spacer_cpu = ChatSpace(device='cpu')
spacer_cpu.space(texts)

Loading JIT Compiled ChatSpace Model


['안녕 안녕 나는 승배야', '어허 한두번이 아니야 경찰서 가고 싶어?']

위의 에러 발생 원인을 규명하고 교정하여 torch에서 gpu사용을 해보자.

# Go!
우선 오류 내용을 보자. `space` 메서드를 사용하면 string일 경우 list로 만들어 batch_texts로 만들고 이를 기반으로 `spacer_iter` 메서드를 호출하여 batch마다 띄어쓰기를 검토한다. space 메서드는 아래와 같다.

```python
def space(self, texts: Union[List[str], str], batch_size: int = 64) -> Union[List[str], str]:
    """
    띄어쓰기 하려는 문장을 넣으면, 띄어쓰기를 수정한 문장을 만들어 줘요!
    전체 문장에 대한 inference가 끝나야 결과가 return 되기 때문에
    띄어쓰기가 되는 순서대로 iterative 하게 사용하고 싶다면 space_iter함수를 하용하세요!

    :param texts: 띄어쓰기를 하고자 하는 문장 또는 문장들
    :param batch_size: 기본으로 64가 설정되어 있지만, 원하는 크기로 조정할 수 있음
    :return: 띄어쓰기가 완료된 문장 또는 문장들
    """

    batch_texts = [texts] if isinstance(texts, str) else texts
    outputs = [output_text for output_text in self.space_iter(batch_texts, batch_size)]
    return outputs if len(outputs) > 1 else outputs[0]
```

다음은 `space_iter` 메서드를 살펴보자.

우선 `data.ChatSpaeDataSet` 객체를 호출하고 이를 `torch.utils.data.DataLoader` 객체에 넣어준 후 `_single_batch_inference` 메서드를 활용하여 batch별 inference를 실시한다. `space_iter`의 코드는 아래와 같다.
```python
def space_iter(self, texts: List[str], batch_size: int = 64) -> Iterable[str]:
    """
    띄어쓰기 하려는 문장을 넣으면, 띄어쓰기를 수정한 문장을 iterative 하게 만들어 줘요!
    모든 띄어쓰기가 끝날 때 까지 기다리지 않아도 되니 for 문에서 사용할 수 있어요.

    내부적으로는 띄어쓰기 하려는 문장(들)을 넣으면 dataset 으로 변환하고
    model.forward에 넣을 수 있도록 token indexing 과 batching 작업을 진행합니다.

    :param texts: 띄어쓰기를 하고자 하는 문장 또는 문장들
    :param batch_size: 기본으로 64가 설정되어 있지만, 원하는 크기로 조정할 수 있음
    :return: 띄어쓰기가 완료된 문장 또는 문장
    :rtype collection.Iterable[str]
    """

    dataset = ChatSpaceDataset(self.config, texts, self.vocab)
    data_loader = DataLoader(dataset, batch_size, collate_fn=dataset.eval_collect_fn)

    for i, batch in enumerate(data_loader):
        batch_texts = texts[i * batch_size : i * batch_size + batch_size]
        for text in self._single_batch_inference(batch=batch, batch_texts=batch_texts):
            yield text
```

이를 불러오는 코드는 아래와 같다.

In [106]:
import json
import re
# typing module, https://michigusa-nlp.tistory.com/3
from typing import Dict, Generator, Iterable, List, Union

import torch
import torch.nn as nn
from torch.utils.data import DataLoader

Inference.py의 다양한 객체 및 PATH 가져오기

In [107]:
# ./data/corpus/
class DynamicCorpus:
    def __init__(self, corpus_path, corpus_size=None, repeat=False):
        self.corpus_path = corpus_path
        self.corpus_file = self.open()
        self.corpus_size = corpus_size if corpus_size else self._get_corpus_size()
        self.line_pointer = 0
        self.repeat = repeat

    def __getitem__(self, item):
        if self.line_pointer >= self.corpus_size:
            if self.repeat:
                self.reload()
            else:
                raise IndexError
        return self.read_line()

    def __iter__(self):
        self.reload()
        for _ in range(self.corpus_size):
            yield self.read_line()

    def __len__(self):
        return self.corpus_size

    def read_line(self):
        line = self.corpus_file.readline().strip()
        self.line_pointer += 1
        return line

    def open(self):
        self.corpus_file = open(self.corpus_path)
        return self.corpus_file

    def reload(self):
        self.corpus_file.close()
        self.open()
        self.line_pointer = 0

    def _get_corpus_size(self):
        line_count = 0
        for _ in self.corpus_file:
            line_count += 1
        self.reload()
        return line_count

In [108]:
import random
from typing import List, Union, Optional, Tuple

from torch.utils.data import Dataset

from collections import Counter

In [109]:
DEFAULT_FORWARD_SPECIAL_TOKENS = ("[PAD]", "[UNK]", "[SOS]", "[EOS]", " ")

In [110]:
# ./vocab/Vocab
class Vocab(dict):
    def __init__(
        self,
        forward_special_tokens: Optional[
            Union[List[str], Tuple[str, ...]]
        ] = DEFAULT_FORWARD_SPECIAL_TOKENS,
        tokens: Optional[Union[List[str], Tuple[str, ...]]] = None,
        backward_special_tokens: Optional[Union[List[str], Tuple[str, ...]]] = None,
    ):
        """
        default forward special tokens will be allocated at forward index (0...4)
        then tokens (list or tuple or iterable of str) will be allocated as next
        at last, special_tokens will be allocated at last position.

        :param tokens: list or tuple of tokens(str)
        :param forward_special_tokens: tuple of string,
                which indicate the special tokens appended in forward
        :param backward_special_tokens: tuple of string,
                which indicate the special tokens appended in backward
        """
        super().__init__()

        self.idx_to_token = []
        self.forward_special_token = forward_special_tokens
        self.tokens = tokens
        self.backward_special_tokens = backward_special_tokens

        # add special tokens on back size
        if forward_special_tokens:
            for special_token in forward_special_tokens:
                self.add(special_token)

        if tokens is not None:
            for token in tokens:
                self.add(token)

        if backward_special_tokens is not None:
            for special_token in backward_special_tokens:
                self.add(special_token)

    def add(self, token: str, index: int = None) -> int:
        """
        add token on this vocab.

        :param token: token which you want to add on this vocab
        :param index: optional, index where you want to add.
            if another token is exist in index, override it into this token.
        :return: index of the token in vocab
        """
        if token in self:
            return self[int]

        token_index = len(self) if index is None else index

        if token_index == len(self) or token_index == 0:
            self.idx_to_token.append(token)
        else:
            self.idx_to_token[token_index] = token

        self[token] = token_index

        return token_index

    def build(
        self,
        lines: Union[Iterable, List[str]],
        min_count: Optional[int] = None,
        max_vocab_size: Optional[int] = None,
        sep_token: str = "",
    ):
        """
        build vocab with multiple string lines.
        vocab will be created as ascending order of token counter.
        however index will be allocated after current fixed special tokens or others.

        :param lines: texts or string iterable
        :param min_count: optional, tokens need to be occurred then min_count
        :param max_vocab_size: optional, vocab size will be limited with max_vocab_size
        :param sep_token: line will be separated by sep_token (default: space)
        :return: vocab instance
        """

        counter = Counter()
        for line in lines:
            for token in line.strip().split(sep_token):
                counter.update(token)

        return self.build_with_counter(counter, min_count, max_vocab_size)

    def build_with_counter(
        self,
        token_counter: Counter,
        min_count: Optional[int] = None,
        max_vocab_size: Optional[int] = None,
    ) -> "Vocab":
        """
        build vocab with token counter.
        vocab will be created as ascending order of counter.
        however index will be allocated after current fixed special tokens or others.

        :param token_counter: counter class instance of token count
        :param min_count: optional, tokens need to be occurred then min_count
        :param max_vocab_size: optional, vocab size will be limited with max_vocab_size
        :return vocab instance
        """

        max_vocab_size = len(token_counter) if max_vocab_size is None else max_vocab_size
        max_vocab_size -= len(self)

        for token, count in token_counter.most_common(max_vocab_size):
            if min_count is None or count >= min_count:
                self.add(token)

        return self

    def get_token(self, index: int) -> str:
        """
        return token of given index.

        :param index: query index to get the token
        :return: token string
        """
        return self.idx_to_token[index]

    @staticmethod
    def load(path: str, with_forward_special_tokens: bool = False) -> "Vocab":
        """
        load vocab file from txt file.

        :param path: vocab txt file path
        :param with_forward_special_tokens:
            if true, the forward special tokens(PAD, EOS ..) will be added
            before txt vocab loading. and txt vocabs will be assigned in
            backward position (e.x apple 4, banana 5 ...)
        :return:
        """

        with open(path) as f:
            tokens = [line.strip() for line in f]

        if with_forward_special_tokens:
            return Vocab(tokens=tokens)
        return Vocab(forward_special_tokens=None, tokens=tokens)

    def dump(
        self,
        path: Optional[str] = None,
        with_forward_special_tokens: bool = True,
        with_backward_special_tokens: bool = True,
    ) -> List[str]:
        """
        dump and save vocab into file

        :param path: path to save vocab
        :param with_forward_special_tokens:
            if true, dumped tokens include the forward_special_token
        :param with_backward_special_tokens:
            if true, dumped tokens include the backward_special_token
        :return: dumped tokens as list of string
        """
        dump_tokens = []

        if with_forward_special_tokens and self.forward_special_token:
            dump_tokens.extend(self.forward_special_token)

        if self.tokens:
            dump_tokens.extend(self.tokens)

        if with_backward_special_tokens and self.backward_special_tokens:
            dump_tokens.extend(self.backward_special_tokens)

        if path is not None:
            with open(path, "w") as f:
                for token in dump_tokens:
                    f.write(f"{token}\n")

        return dump_tokens

    def keys(self):
        return self.idx_to_token

In [111]:
# ./indexer/Indexer
class Indexer:
    """Indexer for word."""

    def __init__(self, vocab: Vocab):
        """
        Init WordIndexer with inherited Indexer initializer

        :param vocab: word vocab object
        """
        self.vocab = vocab

    def encode(
        self,
        text: str,
        max_seq_len: Optional[int] = None,
        min_seq_len: Optional[int] = None,
        pad_idx: int = 0,
        unk_word: str = "[UNK]",
    ) -> Union[List[int], str]:
        """
        Convert tokenized tokens to corresponding ids.

        When `max_seq_len` is not None, encoded tokens will be
        padded up by padding idx to `max_seq_len`.

        :param text: single text for encoding
        :param min_seq_len: minimum sequence length of encoded tokens
            if encoded tokens are shorter then min_seq_len add padding
            tokens (min_seq_len - encoded_token_length) times.
        :param max_seq_len : maximum sequence length of encoded tokens.
            if encoded tokens are longer then max_seq_len then slice it.
        :param pad_idx: padding token index(int) for padding
        :param unk_word: get unk index with unk_word word as key
        :return: list of token ids (int)
        """
        unk_token_id = self.vocab.get(unk_word)
        encoded_text = [self.vocab.get(token, unk_token_id) for token in text]
        if min_seq_len is not None and len(encoded_text) < min_seq_len:
            encoded_text.extend((pad_idx,) * (min_seq_len - len(encoded_text)))
        if max_seq_len:
            encoded_text = (
                encoded_text if len(encoded_text) < max_seq_len else encoded_text[:max_seq_len]
            )
        return encoded_text

    def decode(self, token_ids: Iterable[int], pad_idx: int = 0, as_str=False) -> List[str]:
        """
        Convert token ids to corresponding tokens.

        :param token_ids: token ids(list of int) for decoding
        :param pad_idx: padding token index for padding
        :param as_str: if true, return as concatenated string
            if false, return as list of token string
        :return: return decoded result. return type changed depends on as_str
        """
        decoded_token_ids = [self.vocab.get_token(token_id) for token_id in token_ids]
        return "".join(decoded_token_ids) if as_str else decoded_token_ids

In [112]:
# ./dataset/ChatSpaceDataSet
class ChatSpaceDataset(Dataset):
    def __init__(
        self,
        config,
        texts: Union[DynamicCorpus, List[str]],
        vocab: Vocab,
        with_random_space: bool = False,
    ):
        self.texts = texts
        self.indexer = Indexer(vocab)
        self.space_prob = config["space_prob"]
        self.with_random_space = with_random_space
        self.config = config
        self.lines = []

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        if self.with_random_space:
            model_input = self.get_train_input(self.texts[idx], idx)
        else:
            model_input = {"input": list(self.texts[idx])}

        model_input["input"] = self.indexer.encode(
            model_input["input"], min_seq_len=self.config["min_seq_len"], unk_word="[UNK]"
        )
        model_input["length"] = len(model_input["input"])
        return model_input

    def get_train_input(self, input_text, idx):
        input_char, label = [], []
        word_list = input_text.split()

        for word in word_list:
            word_label = [1] * (len(word) - 1) + [2]
            char_list = list(word)

            if random.random() < self.space_prob[idx % len(self.space_prob)]:
                char_list.append(" ")
                word_label.append(1)

            input_char.extend(char_list)
            label.extend(word_label)

        return {"input": "".join(input_char), "label": label}

    @staticmethod
    def train_collect_fn(batch):
        max_seq_len = max([model_input["length"] for model_input in batch])
        for example in batch:
            example["input"].extend([0] * (max_seq_len - len(example["input"])))
            example["label"].extend([0] * (max_seq_len - len(example["label"])))

        batch = {key: torch.tensor([example[key] for example in batch]) for key in batch[0].keys()}
        return batch

    @staticmethod
    def eval_collect_fn(batch):
        max_seq_len = max([model_input["length"] for model_input in batch])
        for example in batch:
            example["input"].extend([0] * (max_seq_len - len(example["input"])))

        batch = {key: torch.tensor([example[key] for example in batch]) for key in batch[0].keys()}
        return batch

`ChatSpace` 객체를 호출하면 `__init__`에서 아래의 config과 vocab을 호출한다.

현재는 각각의 객체를 전부 메모리에 올려서 어떤 값을 가지는지 확인해보자.

In [113]:
import os

In [114]:
# RESOURCE_PATH = os.path.dirname(os.path.realpath(__file__))
RESOURCE_PATH = 'C:/Users/jinma/Anaconda3/envs/basic/Lib/site-packages/chatspace/resource'

VOCAB_PATH = os.path.join(RESOURCE_PATH, "vocab.txt")
MODEL_DICT_PATH = os.path.join(RESOURCE_PATH, "model/model.pt")
JIT_MODEL_PATH = os.path.join(RESOURCE_PATH, "model/model_jit.pt")
CONFIG_PATH = os.path.join(RESOURCE_PATH, "config.json")

In [115]:
with open(CONFIG_PATH) as f:
    config = json.load(f)

with open(VOCAB_PATH, encoding='utf-8') as f:
    vocab_tokens = [line.strip() for line in f]
vocab = Vocab(tokens=vocab_tokens)
config['vocab_size'] = len(vocab)

In [116]:
config

{'learning_rate': 0.001,
 'batch_size': 512,
 'valid_ratio': 0.025,
 'epochs': 15,
 'vocab_min_freq': 3,
 'min_seq_len': 7,
 'num_workers': 0,
 'logging_step': 500,
 'validation_step': 1500,
 'test_threshold': 0.5,
 'use_multi_gpu': False,
 'device': 'cuda',
 'embedding_dim': 256,
 'cnn_filter': 3,
 'cnn_features': 128,
 'space_prob': [0, 0.15, 0.4],
 'lstm_bidirectional': True,
 'dropout_keep_prob': 0.1,
 'apply_weight': False,
 'lstm_layers': 1,
 'vocab_size': 3563}

In [117]:
vocab

{'[PAD]': 0,
 '[UNK]': 1,
 '[SOS]': 2,
 '[EOS]': 3,
 ' ': 4,
 '<pad>': 5,
 '<unk>': 6,
 '<space>': 7,
 '.': 8,
 '이': 9,
 '어': 10,
 '아': 11,
 '?': 12,
 '다': 13,
 '나': 14,
 '가': 15,
 '그': 16,
 '요': 17,
 'ㅋ': 18,
 '고': 19,
 '지': 20,
 '는': 21,
 '하': 22,
 '니': 23,
 '도': 24,
 '응': 25,
 'ㅎ': 26,
 '자': 27,
 '거': 28,
 '해': 29,
 '에': 30,
 '야': 31,
 '게': 32,
 '서': 33,
 '안': 34,
 '오': 35,
 '데': 36,
 '기': 37,
 'ㅠ': 38,
 '내': 39,
 '있': 40,
 '리': 41,
 '!': 42,
 '시': 43,
 '은': 44,
 '사': 45,
 '네': 46,
 '한': 47,
 '라': 48,
 '일': 49,
 '제': 50,
 '래': 51,
 '구': 52,
 '면': 53,
 '보': 54,
 ',': 55,
 '마': 56,
 '잘': 57,
 '을': 58,
 '만': 59,
 '너': 60,
 '까': 61,
 '먹': 62,
 '뭐': 63,
 '말': 64,
 '왜': 65,
 '난': 66,
 '들': 67,
 '로': 68,
 '무': 69,
 '으': 70,
 '웅': 71,
 '여': 72,
 '저': 73,
 '좀': 74,
 '우': 75,
 '금': 76,
 '수': 77,
 '대': 78,
 '없': 79,
 '좋': 80,
 '겠': 81,
 '주': 82,
 '진': 83,
 '러': 84,
 '와': 85,
 '더': 86,
 '했': 87,
 '할': 88,
 '정': 89,
 '같': 90,
 '알': 91,
 '집': 92,
 '려': 93,
 '음': 94,
 '전': 95,
 '인': 96,
 '럼': 97,


device를 default인 cpu로 돌리면 잘 돌아간다.

이 debugging의 목적은 gpu를 활용하여 speed-up하는 것이다.

In [161]:
custom_device = torch.device('cuda:0')
device = custom_device

In [162]:
device

device(type='cuda', index=0)

In [163]:
# jit를 이용해서 모델 로딩이 가능한 pytorch 버전인지 체크
# string으로 되어있는 torch versino을 비교할 수 있도록 int로 변환

# 제 pc는 120이 나옵니다.
from_jit = int("".join(re.findall(r"[0-9]+", torch.__version__))) >= 110
model_path = JIT_MODEL_PATH if from_jit else MODEL_DICT_PATH

model_path

'C:/Users/jinma/Anaconda3/envs/basic/Lib/site-packages/chatspace/resource\\model/model_jit.pt'

위의 model_path를 통해 model을 호출한다. 호출할 때 `from_jit`이 `True`이면 `_load_model_from_jit`을 사용하고 `RuntimeError`가 발생하면 `_load_model_from_dict`메서드를 활용하여 입력된 device 값을 사용한다. `from_jit`이 `False`이면 바로 `_load_model_from_dict`메서드로 활용한다. (아니, 이럴거면 왜 device를 입력받은거지? default는 True인데...) 코드는 아래와 같다.

```python
if from_jit:
    try:
        model = self._load_model_from_jit(model_path)
    except RuntimeError:
        print("Failed to load jit compiled model. Please set ChatSpace(as_jit=False)")
        model = self._load_model_from_dict(model_path, device)
else:
    model = self._load_model_from_dict(model_path, device)
return model.to(device)
```

바로 한번 호출해보자.

아, 하기에 앞서 `ChatSpaceModel`을 호출해야 한다.

In [164]:
# ./model/components/char_conv
class CharConvolution(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.conv1 = nn.Conv1d(
            in_channels=config["embedding_dim"],
            out_channels=config["cnn_features"],
            kernel_size=config["cnn_filter"],
        )
        self.conv2 = nn.Conv1d(
            in_channels=config["cnn_features"],
            out_channels=config["cnn_features"],
            kernel_size=config["cnn_filter"] * 2 + 1,
        )
        self.conv3 = nn.Conv1d(
            in_channels=config["cnn_features"] * 2,
            out_channels=config["cnn_features"],
            kernel_size=config["cnn_filter"] * 2 - 1,
        )

        self.padding_1 = nn.ConstantPad1d(1, 0)
        self.padding_2 = nn.ConstantPad1d(3, 0)
        self.padding_3 = nn.ConstantPad1d(2, 0)

    def forward(self, embed_input):
        embed_input = torch.transpose(embed_input, dim0=1, dim1=2)
        conv1_output = self.conv1(embed_input)
        conv1_paded = self.padding_1(conv1_output)
        conv2_output = self.conv2(conv1_paded)
        conv2_paded = self.padding_2(conv2_output)
        conv3_input = torch.cat((conv1_paded, conv2_paded), dim=1)
        conv3_output1 = self.conv3(conv3_input)
        conv3_paded1 = self.padding_3(conv3_output1)
        conv3_output2 = self.conv3(embed_input)
        conv3_paded2 = self.padding_3(conv3_output2)
        return torch.cat((conv3_paded1, conv3_paded2, conv3_input), dim=1)
    
# ./model/components/char_lstm
class CharLSTM(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size=config["cnn_features"] // 2,
            hidden_size=config["cnn_features"] // 2,
            num_layers=config["lstm_layers"],
            bidirectional=config["lstm_bidirectional"],
            batch_first=True,
        )

    def forward(self, x, length):
        return self.lstm(x)[0]
    
# ./model/components/embed
class CharEmbedding(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.embedding = nn.Embedding(
            num_embeddings=config["vocab_size"], embedding_dim=config["embedding_dim"]
        )

    def forward(self, input_seq):
        return self.embedding(input_seq)
    
# ./model/components/time_distributed
class TimeDistributed(nn.Module):
    def __init__(self, layer, activation="relu"):
        super().__init__()
        self.layer = layer
        self.activation = self.select_activation(activation)

    def forward(self, x):
        x_reshaped = x.contiguous().view(-1, x.size(-1))

        y = self.layer(x_reshaped)
        y = self.activation(y)

        y = y.contiguous().view(x.size(0), -1, y.size(-1))
        return y

    def select_activation(self, activation):
        if activation == "relu":
            return nn.ReLU()
        elif activation == "sigmoid":
            return nn.Sigmoid()
        elif activation == "tanh":
            return nn.Tanh()
        raise KeyError
        
# ./model/components/projection
class Projection(nn.Module):
    def __init__(self, config):
        super().__init__()
        # 0: PAD_TARGET, 1: NONE_SPACE_TARGET, 2: SPACE_TARGET
        self.seq_fnn = TimeDistributed(nn.Linear(config["cnn_features"], 3))
        self.softmax = nn.LogSoftmax(dim=-1)

    def forward(self, x):
        # feed into projection layer
        x = torch.transpose(x, 1, 0)
        x = self.seq_fnn(x)

        # log-softmax output
        x = torch.transpose(x, 1, 0)
        x = self.softmax(x)
        return x
    
# ./model/components/seq_fnn
class SequentialFNN(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.time_distributed_1 = TimeDistributed(
            nn.Linear(in_features=config["cnn_features"] * 4, out_features=config["cnn_features"])
        )
        self.time_distributed_2 = TimeDistributed(
            nn.Linear(in_features=config["cnn_features"], out_features=config["cnn_features"] // 2)
        )

    def forward(self, conv_embed):
        x = torch.transpose(conv_embed, 2, 1)
        x = self.time_distributed_1(x)
        x = self.time_distributed_2(x)
        return x

In [165]:
# ./model/model
class ChatSpaceModel(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.embed = CharEmbedding(config)
        self.conv = CharConvolution(config)
        self.lstm = CharLSTM(config)
        self.projection = Projection(config)
        self.fnn = SequentialFNN(config)
        self.batch_normalization = nn.BatchNorm1d(4 * config["cnn_features"])
        self.layer_normalization = nn.LayerNorm(config["cnn_features"])

    def forward(self, input_seq, length) -> torch.Tensor:
        x = self.embed.forward(input_seq)
        x = self.conv.forward(x)
        x = self.batch_normalization.forward(x)
        x = self.fnn.forward(x)
        x = self.lstm.forward(x, length)
        x = self.layer_normalization(x)
        x = self.projection.forward(x)
        return x

In [166]:
# model 호출(from jit)
# device = torch.device #  메서드의 기본값으로 실시해본다. <- 여기서 에러가 떴던것..
print("Loading JIT Compiled ChatSpace Model")
model = torch.jit.load(model_path)

Loading JIT Compiled ChatSpace Model


In [168]:
device

device(type='cuda', index=0)

In [169]:
model # Union[torch.jit.ScriptModule, nn.Module]

ScriptModule(
  (embed): ScriptModule(
    (embedding): ScriptModule()
  )
  (conv): ScriptModule(
    (conv1): ScriptModule()
    (conv2): ScriptModule()
    (conv3): ScriptModule()
    (padding_1): ScriptModule()
    (padding_2): ScriptModule()
    (padding_3): ScriptModule()
  )
  (lstm): ScriptModule(
    (lstm): ScriptModule()
  )
  (projection): ScriptModule(
    (seq_fnn): ScriptModule(
      (layer): ScriptModule()
      (activation): ScriptModule()
    )
    (softmax): ScriptModule()
  )
  (fnn): ScriptModule(
    (time_distributed_1): ScriptModule(
      (layer): ScriptModule()
      (activation): ScriptModule()
    )
    (time_distributed_2): ScriptModule(
      (layer): ScriptModule()
      (activation): ScriptModule()
    )
  )
  (batch_normalization): ScriptModule()
  (layer_normalization): ScriptModule()
)

성공적으로 모델을 호출했다. 여기서 끝나는 것이 아니라 `__init__` 단계에서 `model.eval()`메서드를 호출하는 것이 마지막 단계이다.

In [170]:
model.eval()

ScriptModule(
  (embed): ScriptModule(
    (embedding): ScriptModule()
  )
  (conv): ScriptModule(
    (conv1): ScriptModule()
    (conv2): ScriptModule()
    (conv3): ScriptModule()
    (padding_1): ScriptModule()
    (padding_2): ScriptModule()
    (padding_3): ScriptModule()
  )
  (lstm): ScriptModule(
    (lstm): ScriptModule()
  )
  (projection): ScriptModule(
    (seq_fnn): ScriptModule(
      (layer): ScriptModule()
      (activation): ScriptModule()
    )
    (softmax): ScriptModule()
  )
  (fnn): ScriptModule(
    (time_distributed_1): ScriptModule(
      (layer): ScriptModule()
      (activation): ScriptModule()
    )
    (time_distributed_2): ScriptModule(
      (layer): ScriptModule()
      (activation): ScriptModule()
    )
  )
  (batch_normalization): ScriptModule()
  (layer_normalization): ScriptModule()
)

드디어 초기 작업이 끝났군... 이제 다시 `space` 메서드를 보자... 휴

리마인드를 위해 `space`와 `space_iter` 메서드를 여기에 다시 기록한다.

```python
def space(self, texts: Union[List[str], str], batch_size: int = 64) -> Union[List[str], str]:
    """
    띄어쓰기 하려는 문장을 넣으면, 띄어쓰기를 수정한 문장을 만들어 줘요!
    전체 문장에 대한 inference가 끝나야 결과가 return 되기 때문에
    띄어쓰기가 되는 순서대로 iterative 하게 사용하고 싶다면 space_iter함수를 하용하세요!

    :param texts: 띄어쓰기를 하고자 하는 문장 또는 문장들
    :param batch_size: 기본으로 64가 설정되어 있지만, 원하는 크기로 조정할 수 있음
    :return: 띄어쓰기가 완료된 문장 또는 문장들
    """

    batch_texts = [texts] if isinstance(texts, str) else texts
    outputs = [output_text for output_text in self.space_iter(batch_texts, batch_size)]
    return outputs if len(outputs) > 1 else outputs[0]

def space_iter(self, texts: List[str], batch_size: int = 64) -> Iterable[str]:
    """
    띄어쓰기 하려는 문장을 넣으면, 띄어쓰기를 수정한 문장을 iterative 하게 만들어 줘요!
    모든 띄어쓰기가 끝날 때 까지 기다리지 않아도 되니 for 문에서 사용할 수 있어요.

    내부적으로는 띄어쓰기 하려는 문장(들)을 넣으면 dataset 으로 변환하고
    model.forward에 넣을 수 있도록 token indexing 과 batching 작업을 진행합니다.

    :param texts: 띄어쓰기를 하고자 하는 문장 또는 문장들
    :param batch_size: 기본으로 64가 설정되어 있지만, 원하는 크기로 조정할 수 있음
    :return: 띄어쓰기가 완료된 문장 또는 문장
    :rtype collection.Iterable[str]
    """

    dataset = ChatSpaceDataset(self.config, texts, self.vocab)
    data_loader = DataLoader(dataset, batch_size, collate_fn=dataset.eval_collect_fn)

    for i, batch in enumerate(data_loader):
        batch_texts = texts[i * batch_size : i * batch_size + batch_size]
        for text in self._single_batch_inference(batch=batch, batch_texts=batch_texts):
            yield text
```

In [171]:
# 지금은 list이기 때문에 무용지물
batch_texts = [texts] if isinstance(texts, str) else texts

In [172]:
batch_size = 64

In [173]:
# space_iter 메서드
dataset = ChatSpaceDataset(config, batch_texts, vocab)
data_loader = DataLoader(dataset, batch_size, collate_fn=dataset.eval_collect_fn)

In [174]:
data_iter = [(i, batch) for i, batch in enumerate(data_loader)]
data_iter

[(0,
  {'input': tensor([[ 34, 398,  34, 398,  14,  21, 528, 186,  31,   0,   0,   0,   0,   0,
              0,   0,   0],
           [ 10, 299,  47, 125, 170,   9,  11,  23,  31, 247, 622,  33,  15,  19,
            171,  10,  12]]),
   'length': tensor([ 9, 17])})]

In [175]:
i, batch = data_iter[0]
i, batch

(0,
 {'input': tensor([[ 34, 398,  34, 398,  14,  21, 528, 186,  31,   0,   0,   0,   0,   0,
             0,   0,   0],
          [ 10, 299,  47, 125, 170,   9,  11,  23,  31, 247, 622,  33,  15,  19,
           171,  10,  12]]),
  'length': tensor([ 9, 17])})

In [176]:
batch_texts = batch_texts[i * batch_size : i * batch_size + batch_size]
batch_texts

['안녕안녕나는승배야', '어허한두번이아니야경찰서가고싶어?']

현재는 2개의 text만 존재하여 전부 뽑혔지만 batch_size를 설정하기에 따라 한번에 처리할 text의 양이 변하는 구조이다.

batch별 text에 대해 `_single_batch_inference` 메서드를 활용하여 inference를 수행하고 text를 return한다. 해당 메서드는 다음과 같다.

```python
def _single_batch_inference(
    self, batch: Dict[str, torch.Tensor], batch_texts: List[str]
) -> Generator[str, str, None]:
    """
    batch input 을 모델에 넣고, 예측된 띄어쓰기를 원본 텍스트에 반영하여
    띄어쓰기가 완료된 텍스트를 iterative 하게 생성 합니다!

    :param batch: 'input', 'length' 두 키를 갖는 batch input
        input은 char 를 encoding 한 [batch, seq_len] 크기의 torch.LongTensor
        length는 각 sequence 의 길이 정보를 갖고 있는 [batch] 크기의 torch.LongTensor
        length를 사용하는 이유는 dynamic LSTM을 사용하기 위해서 pack_padded_sequence 를 사용하기 때문임

    :param batch_texts: batch 에 들어간 실제 원본 문장들
    :return: 띄어쓰기가 완료된 문장
    :rtype collection.Iterable[str]
    """
    # model forward for chat-space nn.Module
    output = self.model.forward(batch["input"], batch["length"])

    # make probability into class index with argmax
    space_preds = output.argmax(dim=-1).cpu().tolist()

    for text, space_pred in zip(batch_texts, space_preds):
        # yield generated text (spaced text)
        yield self.generate_text(text, space_pred)
```

In [177]:
# model forward for chat-space nn.Module
output = model.forward(batch['input'], batch['length'])

In [178]:
output

tensor([[[-1.2443e+01, -3.3778e-04, -8.0049e+00],
         [-7.0465e+00, -2.5429e+00, -8.2852e-02],
         [-1.1287e+01, -5.8670e-04, -7.4629e+00],
         [-6.9063e+00, -2.9371e+00, -5.5537e-02],
         [-1.0548e+01, -1.5441e-03, -6.4913e+00],
         [-8.1212e+00, -5.0494e+00, -6.7329e-03],
         [-9.5067e+00, -5.8422e-03, -5.1584e+00],
         [-1.0821e+01, -1.5176e-03, -6.5046e+00],
         [-1.2022e+01, -1.1197e+01, -1.9669e-05],
         [ 0.0000e+00, -1.7328e+01, -1.7328e+01],
         [ 0.0000e+00, -2.2758e+01, -2.2758e+01],
         [ 0.0000e+00, -2.2310e+01, -2.2310e+01],
         [ 0.0000e+00, -2.2295e+01, -2.2295e+01],
         [ 0.0000e+00, -2.1146e+01, -2.1146e+01],
         [ 0.0000e+00, -2.0427e+01, -2.0427e+01],
         [ 0.0000e+00, -2.2574e+01, -2.2574e+01],
         [ 0.0000e+00, -2.5446e+01, -2.5446e+01]],

        [[-1.1313e+01, -1.0318e-03, -6.8889e+00],
         [-8.2439e+00, -4.2906e+00, -1.4058e-02],
         [-6.9846e+00, -2.2570e-01, -1.6039e+00]

응? 여기서 이상한 method를 발견했다. 왜 cpu()를 사용할까..?

In [180]:
space_preds = output.argmax(dim=-1).cpu().tolist()
space_preds

[[1, 2, 1, 2, 1, 2, 1, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 2, 1, 1, 1, 2, 1, 1, 2, 1, 1, 2, 1, 2, 1, 1, 2]]

In [181]:
res = [(text, space_pred) for text, space_pred in zip(batch_texts, space_preds)]
res

[('안녕안녕나는승배야', [1, 2, 1, 2, 1, 2, 1, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0]),
 ('어허한두번이아니야경찰서가고싶어?', [1, 2, 1, 1, 1, 2, 1, 1, 2, 1, 1, 2, 1, 2, 1, 1, 2])]

이 다음엔 prediction된 class index를 `generate_text` 메서드를 활용, 실제 띄어쓰기로 generation한다. 코드는 아래와 같다.
```python 
def generate_text(self, text: str, space_pred: List[int]) -> str:
    """
    prediction 된 class index 를 실제 띄어쓰기로 generation 하는 부분

    :param text: 띄어쓰기가 옳바르지 않은 원본 문장
    :param space_pred: ChatSpaceModel.forward 에서 나온 결과를
    argmax(dim=-1)한 [batch, seq_len] 크기의 3-class torch.LongTensor
    0: PAD_TARGET, 1: NONE_SPACE_TARGET, 2: SPACE_TARGET
    :return: 띄어쓰기가 반영된 문장
    """
    generated_sentence = list()
    for i in range(len(text)):
        if space_pred[i] - 1 == 1:
            generated_sentence.append(text[i] + " ")
        else:
            generated_sentence.append(text[i])

    joined_chars = "".join(generated_sentence)
    return re.sub(r" {2,}", " ", joined_chars).strip()
```

In [182]:
generated_sentence = list()
text, space_pred = res[0]
for i in range(len(text)):
    if space_pred[i] - 1 == 1:
        generated_sentence.append(text[i] + " ")
    else:
        generated_sentence.append(text[i])
print(text)
generated_sentence

안녕안녕나는승배야


['안', '녕 ', '안', '녕 ', '나', '는 ', '승', '배', '야 ']

In [183]:
joined_chars = "".join(generated_sentence)
joined_chars

'안녕 안녕 나는 승배야 '

In [184]:
re.sub(r" {2,}", " ", joined_chars).strip()

'안녕 안녕 나는 승배야'

???????????? 왜 직접 코드를 따라가니까 굉장히 잘되는거지...?

에러가 발생한 부분은 `_single_batch_inference` 메서드에서 `forward`하는 부분이었다.

지금 혹시 CPU로 돌리고 있나..?

CPU로 돌리고 있는지 아닌지 다량의 데이터를 넣어 시간 체크를 해보자.

In [185]:
import pandas as pd

data_path = './data/' # 각자의 data path를 입력하도록 하자.
df_train = pd.read_csv(data_path + 'train.csv')
df_test = pd.read_csv(data_path + 'public_test.csv')

In [186]:
df_train.head()

,id,year_month,text,smishing
0,0,2017-01,XXX은행성산XXX팀장입니다.행복한주말되세요,0
1,1,2017-01,오늘도많이웃으시는하루시작하세요XXX은행 진월동VIP라운지 XXX올림,0
2,2,2017-01,안녕하십니까 고객님. XXX은행입니다.금일 납부하셔야 할 금액은 153600원 입니...,0
3,4,2017-01,XXX 고객님안녕하세요XXX은행 XXX지점입니다지난 한 해 동안 저희 XXX지점에 ...,0
4,5,2017-01,1월은 새로움이 가득XXX입니다.올 한해 더 많이행복한 한해되시길바랍니다,0


In [187]:
df_test.head()

,id,year_month,text
0,340000,2019-01,XXX고객님! 안녕하세요? 새롭게 시작하는 한 주 행복 가득하시길 기원합니다. 지난...
1,340001,2019-01,긴급 안내 XXX은행 가락동 지점 - 헬리오XXX 기본XXX 대출이자를 ...
2,340002,2019-01,XXX 고객님 안녕하세요올해는 미세먼지가 유난인거 같습니다.엊그제 새해가 시작된거같...
3,340003,2019-01,XXX 고객님찾아온 행운을 잡으셨나요? 못잡으셨다면 이번에 다시 잡으시길 기원합니다...
4,340004,2019-01,XXX 고객님새해 복 많이 받으세요 XXX은행 코스트코 퇴직연금 담당자입니다. 고...


In [188]:
exam_texts = df_train.loc[100:500, 'text'].tolist()

In [189]:
spacer_cpu = ChatSpace(device='cpu') # bench-mark

Loading JIT Compiled ChatSpace Model


In [ ]:
class ChatSpace_Mine:
    
    def __init__(self, config, vocab, device):
        self.config = config
        self.vocab = vocab
        self.device = torch.device(device)

        if model_path is None:
            from_jit = self._is_jit_available() if from_jit else False
            model_path = JIT_MODEL_PATH if from_jit else MODEL_DICT_PATH

        self.model = self._load_model(model_path, self.device, from_jit=from_jit)
        self.model.eval()